# I. Load Scores via ESPN API

In [1]:
from espn_functions import *
#Define week and year
week = 4
year = 2023
standings_df, matchup_df, HP_Owner, HP_Player, HP_Score, HT_Owner, HT_Score = run_espn_weekly(week, year)

Data not loaded correctly (Attempt 1 of 5), retrying in 1 second...
Loaded data from ESPN for week 4 of the 2023 season
Completed processing weekly scores and standings


In [7]:
espn_cookies = {
    "swid": config_data['espn']['swid'],
    "espn_s2": config_data['espn']['espn_s2']
}

league_id = config_data['espn']['league_id']
league_data = load_league(league_id, espn_cookies, year)
owner_dict = config_data['espn']['owner_dict']

league_id = config_data['espn']['league_id']
league_data = load_league(league_id, espn_cookies, year)
owner_dict = config_data['espn']['owner_dict']

attempts = 0
max_attempts=5
while attempts < max_attempts:
    #Load the league data via the ESPN API
    standings_df = load_records(league_data)
    weekly_df = load_weekly_stats(year, league_id, espn_cookies, week)
    schedule_df = load_schedule(year, league_id, espn_cookies, week)

    matchup_df = transform_weekly(weekly_df, schedule_df)
    standings_df = iterate_weeks_espn(year, week, standings_df, league_id, espn_cookies)
    standings_df = rank_playoff_seeds(standings_df)
    matchup_df = standings_df[['teamId', 'team_name']].merge(matchup_df, on='teamId', how='left').sort_values(by='matchup_id').reset_index(drop=True)

    if (len(matchup_df) > 0) and (len(schedule_df) > 0) and (len(standings_df) > 0):
        print('Loaded data from ESPN for week ' + str(week) + ' of the ' + str(year) + ' season')
        break  # Break the loop if data is loaded correctly
    else:
        attempts += 1
        print(f'Data not loaded correctly (Attempt {attempts} of {max_attempts}), retrying in 1 second...')
        time.sleep(1)  # Wait for 1


Loaded data from ESPN for week 4 of the 2023 season


# II. Create GPT Summary Email

In [17]:
summary = generate_summary(week, matchup_df_true, standings_df, model = 'gpt-4')
print(summary)

Generated the summary
Hello Punt-Intentional Leaguers,

Another week of nail-biting games is now in the books and I'm here to serve up the deets with a side of good natured ribbing. So, let's get scoring!

Starting off with the top tier, we've got King Drew and The Real Artist sitting pretty undefeated after four weeks. Looks like Andrew's draft strategy and John's auction shenanigans are paying off. Sleep with one eye open, fellas. History has shown us there are no safe leads in this league. 

Also, is Kai Shen in the room? Not Sketch just quietly moved into 3rd place at 3-1. All hail C.J. Stroud and company. Shen's consistent approach to mediocre performance is turning out to be quite the master plan. Everyone, take note. “Barely satisfactory” is the way to go! On a related note, did Richard Liang get the memo? Sadly, he has a 1-3 record, but in the spirit of looking at the glass half-full, at least you're not Richie... 

Moving onto the disappointments of the week, sorry Kyle O'Mear

# III. Send G-Mail

In [19]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']
recipients = list(config_data['emails']['espn_email_dict'].values())

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

#Format the tables
scoring_tabulate = tabulate(matchup_df, headers='keys', tablefmt='html', showindex=False)
standings_tabulate = tabulate(standings_df, headers='keys', tablefmt='html', showindex=False)

# Define message
subject = f"Weekly Fagudas FF Recap: Week {week}"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p><b>Highest Scoring Team</b>: {HT_Owner}: {HT_Score} points</p>
    <p><b>Highest Scoring Player</b>: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings_tabulate}
  </body>
</html>
"""

sender = user

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection z19-20020ae9c113000000b0076daaccb7f7sm456117qki.135 - gsmtp')